In [4]:
'''
Import Packages
'''
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mahotas
import cv2
import os
import h5py

In [51]:
'''
Parameters
'''
images_per_class       = 10
fixed_size             = tuple((500, 500))
train_path             = "C:\\Users\\ganesh appu\\Downloads\\project_imageprocessing\\Plant_leave_diseases_dataset_with_augmentation\\"
h5_train_data          = 'C:\\Users\\ganesh appu\\Downloads\\project_imageprocessing\\train_data.h5'
h5_train_labels        = 'C:\\Users\\ganesh appu\\Downloads\\project_imageprocessing\\train_labels.h5'
bins                   = 8

In [52]:
'''
function for convertion of Image format from BGR TO RGB
'''
def rgb_bgr(image):
    rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return rgb_img

In [53]:
'''
Converting the Image from RGB to HSV color space
'''
def bgr_hsv(rgb_img):
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)
    return hsv_img

In [54]:
'''
Image Segmentation
'''
def img_segmentation(rgb_img,hsv_img):
    lower_green = np.array([25,0,20])
    upper_green = np.array([100,255,255])
    healthy_mask = cv2.inRange(hsv_img, lower_green, upper_green)
    result = cv2.bitwise_and(rgb_img,rgb_img, mask=healthy_mask)
    lower_brown = np.array([10,0,10])
    upper_brown = np.array([30,255,255])
    disease_mask = cv2.inRange(hsv_img, lower_brown, upper_brown)
    disease_result = cv2.bitwise_and(rgb_img, rgb_img, mask=disease_mask)
    final_mask = healthy_mask + disease_mask
    final_result = cv2.bitwise_and(rgb_img, rgb_img, mask=final_mask)
    return final_result

In [55]:
'''
Global Feature: 1 Shape
Hu moments
'''
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [56]:
'''
Global Feature: 2 Texture
Haralick Texture
'''
def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

In [57]:
'''
Global Feature: 3 Color
'''
def fd_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [58]:
train_labels = os.listdir(train_path)
train_labels.sort()
print(train_labels)
global_features = []
labels          = []

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Background_without_leaves', 'Blueberry___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl

In [59]:
pwd

'C:\\Users\\ganesh appu\\Downloads\\project_imageprocessing'

In [60]:
for training_name in train_labels:
    dir = os.path.join(train_path, training_name)

    current_label = training_name
    #Images in each file
    for x in range(1,images_per_class+1):
        file = dir + "\\" + "image (" +str(x) + ")" +".jpg"

        # Reading the image and converting it to required size
        image = cv2.imread(file)
        image = cv2.resize(image, fixed_size)

        
        # Calling Functions
        
        RGB_BGR       = rgb_bgr(image)
        BGR_HSV       = bgr_hsv(RGB_BGR)
        IMG_SEGMENT   = img_segmentation(RGB_BGR,BGR_HSV)

        # Extracting Features
        
        fv_hu_moments = fd_hu_moments(IMG_SEGMENT)
        fv_haralick   = fd_haralick(IMG_SEGMENT)
        fv_histogram  = fd_histogram(IMG_SEGMENT)
        
        global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
        
        #Storing the features
        labels.append(current_label)
        global_features.append(global_feature)

    print("processed folder: {}".format(current_label))

print("completed Global Feature Extraction...")


processed folder: Apple___Apple_scab
processed folder: Apple___Black_rot
processed folder: Apple___Cedar_apple_rust
processed folder: Apple___healthy
processed folder: Background_without_leaves
processed folder: Blueberry___healthy
processed folder: Cherry___Powdery_mildew
processed folder: Cherry___healthy
processed folder: Corn___Cercospora_leaf_spot Gray_leaf_spot
processed folder: Corn___Common_rust
processed folder: Corn___Northern_Leaf_Blight
processed folder: Corn___healthy
processed folder: Grape___Black_rot
processed folder: Grape___Esca_(Black_Measles)
processed folder: Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
processed folder: Grape___healthy
processed folder: Orange___Haunglongbing_(Citrus_greening)
processed folder: Peach___Bacterial_spot
processed folder: Peach___healthy
processed folder: Pepper,_bell___Bacterial_spot
processed folder: Pepper,_bell___healthy
processed folder: Potato___Early_blight
processed folder: Potato___Late_blight
processed folder: Potato___healthy

In [66]:
'''
Overall vector size used for training the model is
'''
print("No of Features are {}".format(np.array(global_features).shape))

No of Features are (390, 532)


In [67]:
'''
Overall Training Label size
'''
print("Training Labels {}".format(np.array(labels).shape))

Training Labels (390,)


In [68]:
# Encoding the Training Labels to Decrease the size
targetNames = np.unique(labels)
le          = LabelEncoder()
target      = le.fit_transform(labels)
print("Training labels encoded")

Training labels encoded


In [69]:
#Normalizing the feature vectors for size reduction
from sklearn.preprocessing import MinMaxScaler
scaler            = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(global_features)
print("Feature vector normalized")

Feature vector normalized


In [70]:
print("Target labels: {}".format(target))
print("Target labels shape: {}".format(target.shape))

Target labels: [ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2  2
  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4
  4  4  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  7  7
  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9
  9  9  9  9 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 14 14 14 14
 14 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16
 16 16 17 17 17 17 17 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 19 19
 19 19 19 19 19 19 19 19 20 20 20 20 20 20 20 20 20 20 21 21 21 21 21 21
 21 21 21 21 22 22 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 23 23 23
 24 24 24 24 24 24 24 24 24 24 25 25 25 25 25 25 25 25 25 25 26 26 26 26
 26 26 26 26 26 26 27 27 27 27 27 27 27 27 27 27 28 28 28 28 28 28 28 28
 28 28 29 29 29 29 29 29 29 29 29 29 30 30 30 30 30 30 30 30 30 30 31 31
 31 31 31 31 31 31 31 31 32 32 32 32

In [74]:
#To save the feature vector on the local system
h5f_data = h5py.File(h5_train_data, 'w')
h5f_data.create_dataset('dataset#1', data=np.array(rescaled_features))

<HDF5 dataset "dataset#1": shape (390, 532), type "<f8">

In [75]:
#To save target labels
h5f_label = h5py.File(h5_train_labels, 'w')
h5f_label.create_dataset('dataset#1', data=np.array(target))

<HDF5 dataset "dataset#1": shape (390,), type "<i8">

In [76]:
h5f_data.close()
h5f_label.close()